In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/laptops.csv')
df.head()

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


In [3]:
df.isnull().sum()

Laptop             0
Status             0
Brand              0
Model              0
CPU                0
RAM                0
Storage            0
Storage type      42
GPU             1371
Screen             4
Touch              0
Final Price        0
dtype: int64

In [5]:
df = df.drop(labels=['Laptop','Status','Touch','Model'],axis=1)

In [6]:
#independent and dependent variables
X = df.drop(labels=['Final Price'],axis=1)
y =df[["Final Price"]]

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         2160 non-null   object 
 1   CPU           2160 non-null   object 
 2   RAM           2160 non-null   int64  
 3   Storage       2160 non-null   int64  
 4   Storage type  2118 non-null   object 
 5   GPU           789 non-null    object 
 6   Screen        2156 non-null   float64
dtypes: float64(1), int64(2), object(4)
memory usage: 118.2+ KB


In [11]:
#segrate numerial cols and cat cols

categorical_cols = X.select_dtypes(include='object').columns

numerical_cols = X.select_dtypes(exclude='object').columns

In [12]:
categorical_cols

Index(['Brand', 'CPU', 'Storage type', 'GPU'], dtype='object')

In [15]:
categorical_cols = categorical_cols.drop(labels=['Brand'])

In [16]:
categorical_cols

Index(['CPU', 'Storage type', 'GPU'], dtype='object')

In [17]:
numerical_cols

Index(['RAM', 'Storage', 'Screen'], dtype='object')

In [29]:
X['CPU'].unique()

array(['Intel Core i5', 'Intel Celeron', 'Intel Core i3', 'Intel Core i7',
       'AMD Ryzen 5', 'AMD Ryzen 7', 'AMD Ryzen 3', 'Apple M1',
       'AMD Athlon', 'Apple M2', 'AMD Ryzen 9', 'Intel Core i9',
       'AMD 3020e', 'Qualcomm Snapdragon 7', 'Intel Evo Core i7',
       'Intel Evo Core i5', 'Intel Pentium', 'Apple M2 Pro',
       'AMD Radeon 5', 'Intel Evo Core i9', 'AMD 3015e', 'Apple M1 Pro',
       'Intel Core M3', 'AMD Radeon 9', 'Mediatek MT8183', 'AMD 3015Ce',
       'Qualcomm Snapdragon 8', 'Microsoft SQ1'], dtype=object)

In [36]:
#define custom ranking for each cat col

CPU_categories = ['Intel Celeron',
'Microsoft SQ1',
'Qualcomm Snapdragon 7',
'Qualcomm Snapdragon 8',
'Mediatek MT8183',
'AMD 3015e',
'AMD 3015Ce',
'AMD 3020e', 'AMD Athlon',
'Intel Core M3',
'AMD Radeon 5',
'Intel Pentium',
'Apple M2',
'Apple M2 Pro',
'AMD Radeon 9',
'Intel Core i3',
'AMD Ryzen 3',
'Intel Evo Core i5',

'Intel Evo Core i7',
'Intel Evo Core i9',
'AMD Ryzen 5',
'Apple M1',
'Apple M1 Pro',
'Intel Core i5',
'Intel Core i7',
'AMD Ryzen 7',
'Intel Core i9',
'AMD Ryzen 9'
]

Storage_type_categories = [ 'eMMC','SSD']

GPU_categories =['A 370M',
                 '610 M',
'A 730M',
'GTX 1050',
'GTX 1070',
'GTX 1650',
'GTX 1660',
'MX 130',
'MX 330',
'MX 450',
'MX 550',
'P 500',
'Radeon Pro 5300M',
'Radeon Pro 5500M',
'Radeon RX 6600M',
'Radeon Pro RX 560X',
'RX 6500M',
'RX 6600M',
'RX 6700M',
'RX 6800S',
'RX 7600S',
'RTX 2050',
'RTX 2060',
'RTX 2070',
'RTX 2080',
'RTX 3000',
'RTX 3050',
'RTX 3060',
'RTX 3070',
'RTX 3080',
'RTX 4050',
'RTX 4060',
'RTX 4070',
'RTX 4080',
'RTX 4090',
'RTX A1000',
'RTX A2000',
'RTX A3000',
'RTX A5500',
'T 1000',
'T 1200',
'T 2000',
'T 500',
'T 550',
'T 600'
]

In [37]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

#pipline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [38]:
#numerical pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
         
    ]
)

#categorical pipeline

cat_pipeline= Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[CPU_categories,Storage_type_categories,GPU_categories])),
        ('scaler',StandardScaler())

    ]
)

preprocessing  = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.30,random_state=42)

In [40]:
X_train.shape,X_test.shape

((1512, 7), (648, 7))

In [41]:
X_train = pd.DataFrame(preprocessing.fit_transform(X_train),columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(preprocessing.transform(X_test),columns=preprocessing.get_feature_names_out())

In [42]:
X.head()

,Brand,CPU,RAM,Storage,Storage type,GPU,Screen
0,Asus,Intel Core i5,8,512,SSD,NaN,15.6
1,Alurin,Intel Celeron,8,256,SSD,NaN,15.6
2,Asus,Intel Core i3,8,256,SSD,NaN,15.6
3,MSI,Intel Core i7,16,1000,SSD,RTX 3050,15.6
4,HP,Intel Core i5,16,512,SSD,NaN,15.6


In [43]:
X_train.head()

,num_pipeline__RAM,num_pipeline__Storage,num_pipeline__Screen,cat_pipeline__CPU,cat_pipeline__Storage type,cat_pipeline__GPU
0,0.079482,-0.225361,0.341547,-0.146669,0.171089,-0.084640
1,1.760269,1.150205,1.776163,0.988088,0.171089,1.082307
2,0.079482,-0.225361,0.341547,0.501763,0.171089,-0.668113
3,1.760269,1.150205,0.341547,0.501763,0.171089,0.887816
4,0.079482,-0.225361,1.522996,0.501763,0.171089,-1.057095


In [44]:
#model training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [45]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)


array([1287.13433201])

In [46]:
regressor.coef_

array([[398.34932453, 301.45732627, -37.93019865,  43.38423722,
         -5.55971206, 159.47679358]])

In [47]:
regressor.intercept_

array([1287.13433201])

In [48]:
import numpy as np
def evaluate_model(true,predict):
    mae = mean_absolute_error(true,predict)
    mse = mean_squared_error(true,predict)
    rmse= np.sqrt(mean_squared_error(true,predict))
    r2_square = r2_score(true,predict)

    return mae,rmse,r2_square

In [49]:
y_pred = regressor.predict(X_test)

In [51]:
#train multiple models
#model evaluation

models= {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'DecisionTreeRegressor':DecisionTreeRegressor()
}


trained_models_list =[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)


    #make predictions
    y_pred = model.predict(X_test)

    mae,rmse,r2_square = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])

    model_list.append(list(models.keys())[i])

    print("model training performance")

    print("RMSE",rmse)

    print("MAE",mae)

    print("R2_score",r2_square*100)

    r2_list.append(r2_square)

    print("="*35)

    print("\n")

LinearRegression
model training performance
RMSE 589.9402130700781
MAE 426.3533381562334
R2_score 61.612609498411885




Lasso
model training performance
RMSE 589.9897208431416
MAE 426.5369991037785
R2_score 61.60616628969605


Ridge
model training performance
RMSE 589.9464228775508
MAE 426.3853121157355
R2_score 61.61180135021749


ElasticNet
model training performance
RMSE 617.7592455447462
MAE 454.25609272979915
R2_score 57.906881981963835


DecisionTreeRegressor
model training performance
RMSE 493.0866634266528
MAE 304.74620219903125
R2_score 73.18244982957344


